In [1]:
import pandas as pd
from collections import Counter
import unidecode
from nltk.corpus import stopwords
import nltk
import string
import re
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from icecream import ic
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
lemm = WordNetLemmatizer()


In [2]:
stemmer = SnowballStemmer("russian")

morph = MorphAnalyzer()

In [3]:
unidecoded_nospam_words = []
unidecoded_spam_words = []
def unidecoder(phrase):

    return [unidecode.unidecode(word) for word in phrase] 

In [4]:
def second_unidecoder(phrase):

    return unidecode.unidecode(phrase)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kazakov.va\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop_words = stopwords.words()

In [7]:
spam_df = pd.read_excel('спам.xlsx', names=['text'])
spam_df = spam_df.head(20000)
spam_df.drop_duplicates(subset=['text'], inplace=True)
spam_df.shape

(20000, 1)

In [8]:
no_spam_df = pd.read_excel('не_спам.xlsx', names=['id', 'text']).head(20000)
no_spam_df.head()
no_spam_df.drop(no_spam_df.index[0], inplace=True)
no_spam_df.shape


(19999, 2)

In [9]:
def preprocess_text(text):

    if not isinstance(text, str):
        return []
    text = re.sub(r"[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+", " ", text)
    text = re.sub(r"["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", " ", text)

    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text) 
    
    text = text.lower() 
    tokens = word_tokenize(text)
    
    # stemmed_words = [stemmer.stem(word) for word in tokens]
    # ic(stemmed_words)
    # tokens = [word for word in words if word not in unidecoded_stop_words] 
    
    return tokens

In [10]:
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kazakov.va\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kazakov.va\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
spam_df['preprocessed_text'] = spam_df['text'].apply(preprocess_text)
no_spam_df['preprocessed_text'] = no_spam_df['text'].apply(preprocess_text)

In [12]:
def normalizer(text):
    return [morph.normal_forms(word)[0] for word in text]

In [13]:
spam_df['preprocessed_text'] = spam_df['preprocessed_text'].apply(normalizer)
no_spam_df['preprocessed_text'] = no_spam_df['preprocessed_text'].apply(normalizer)


In [14]:
def delete_stopwords(text):

    tokens = [word for word in text if word not in stop_words]  # Remove stopwords
    
    return tokens

In [15]:
no_spam_df['cleaned_text'] = no_spam_df['preprocessed_text'].apply(delete_stopwords)
spam_df['cleaned_text']= spam_df['preprocessed_text'].apply(delete_stopwords)

In [16]:
no_spam_df['unidecoded_text'] = no_spam_df['cleaned_text'].apply(unidecoder)
spam_df['unidecoded_text'] = spam_df['cleaned_text'].apply(unidecoder)

In [17]:
spam_df['label'] = 1
no_spam_df['label'] = 0

In [14]:
unidecoded_nospam_words = [word for word in cleaned_nospam if word not in ['', '-', 'A', 'V', '!!!', '{"type":', 'Vy', 'I', '.',
                                                                                    'g', 'let', 'd', 'p', 'rf', 'te', 'n', 't',  'text', 'type',
                                                                                    
                                                                                    'S', 'Da', '2.', '3.', '4.', '5.', 'eto', 'ul']]

unidecoded_spam_words = [word for word in cleaned_spam if word not in ['', 'you', 'your', 'a', '-', '--' 'A', 'for', 'c', 'to','is',
'and',  'ot', 'k', 'Rs', 'of', 'Withdrawal', 'the',   'withdrawal',    'day', 'in', 'me', 'get', 'this', 'it', 'have', 'can', 'if','he', 'want' , '!!', 'that', 'who', "ect'", 'always'
'ne','trx', 'b', 't', 'i', 'A', 'za', 'all', 'us', '+', 'do', 'p', 'y', 'u', 'C','[A][A][A][A]', '|#|#|#|#|#|#||#||#|#|#|', '|RECHARgE'
'r', 'R', 'V', '!!!', '{"type":', 'Vy', 'I', '.', 'S', 'Da', '2.', '3.', '4.', '5.', 'eto', 'ul', 'na', 'text', 'type']]

In [125]:
df = pd.concat([spam_df, no_spam_df])

In [126]:
df_unidecoded = df[['unidecoded_text', 'label']]

In [127]:
def clean_text(text):
    cleaned_text = [re.sub(r'[,\[\]]', '', word) for word in text if word.strip()]
    return [word for word in cleaned_text if word]  # Убираем пустые строки

cleantest = df_unidecoded['unidecoded_text'].apply(clean_text)

In [128]:
df_unidecoded['joined_text'] = df_unidecoded['unidecoded_text'].apply(lambda x: ' '.join(x) if x else '')

C:\Users\kazakov.va\AppData\Local\Temp\ipykernel_52296\3718935788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unidecoded['joined_text'] = df_unidecoded['unidecoded_text'].apply(lambda x: ' '.join(x) if x else '')


In [136]:
df_unidecoded = df_unidecoded.query('joined_text != ""')



In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [144]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 5), min_df=2, max_features=5000)
X = tfidf_vectorizer.fit_transform(df_unidecoded['joined_text'])
y = df_unidecoded['label']

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [146]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

In [147]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)*100
ic(accuracy)

ic| accuracy: 88.05732484076432


88.05732484076432

In [148]:
my_phrase = """
Супер,УК комсервис бездействует ,там уже стаи крыс по мусору бегают
"""
# 1. Предобработка фразы
# my_phrase_processed = preprocess_text(my_phrase)
# ic(my_phrase_processed)


# 2. Преобразуем список слов обратно в строку
# my_phrase_joined = ' '.join(my_phrase_processed)

# 3. Применяем unidecoder
my_phrase = str(my_phrase).lower()
my_phrase_unidecoded = second_unidecoder(my_phrase)
ic(my_phrase_unidecoded)

# 4. Векторизация фразы
my_phrase_vect = tfidf_vectorizer.transform([my_phrase_unidecoded])
ic(my_phrase_vect)

# 5. Прогнозирование
prediction = nb_classifier.predict(my_phrase_vect)
ic(prediction)

# 6. Результат
if prediction[0] == 1:
    print("Спам")
else:
    print("Не спам")

ic| my_phrase_unidecoded: '
                           super,uk komservis bezdeistvuet ,tam uzhe stai krys po musoru begaiut
                           '
ic| my_phrase_vect: <Compressed Sparse Row sparse matrix of dtype 'float64'
                    	with 3 stored elements and shape (1, 5000)>
ic| prediction: array([1], dtype=int64)


Спам
